## import all the necessary libraries

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

instantiate a spark session

In [2]:
spark = SparkSession.builder.getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/25 18:36:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/25 18:36:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/25 18:36:07 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/03/25 18:36:07 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
df = spark.read.format('json').load('/Users/rob_the_programmer/Documents/programming/2022/hands_on_data_science/spark-sql-dataframes-workspace/files/utlization.json')

In [4]:
df.show()

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
|           0.32|03/05/2019 08:56:14| 

In [5]:
va = VectorAssembler(inputCols=['cpu_utilization', 'free_memory', 'session_count'], outputCol='features')

In [8]:
vectorcluster = va.transform(df)

In [9]:
vectorcluster.show()

+---------------+-------------------+-----------+---------+-------------+----------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|        features|
+---------------+-------------------+-----------+---------+-------------+----------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|[0.57,0.51,47.0]|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|[0.47,0.62,43.0]|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|[0.56,0.57,62.0]|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|[0.57,0.56,50.0]|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|[0.35,0.46,43.0]|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|[0.41,0.58,48.0]|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|[0.57,0.35,58.0]|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58| [0.41,0.4,58.0]|

In [10]:
kmeans = KMeans().setK(3)
kmeans = kmeans.setSeed(1)

In [12]:
kmodel = kmeans.fit(vectorcluster)

In [13]:
kmodel.clusterCenters()

[array([ 0.62881549,  0.37094643, 70.43030159]),
 array([ 0.52047775,  0.47836303, 51.79927162]),
 array([ 0.71931575,  0.28104316, 88.23965784])]

22/03/26 07:07:12 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 7186814 ms exceeds timeout 120000 ms
22/03/26 07:07:19 WARN SparkContext: Killing executors is not supported by current scheduler.


linear regression

In [16]:
va = VectorAssembler(inputCols=['cpu_utilization'], outputCol='features')
dfvectorutil = va.transform(df)
dfvectorutil.show()

+---------------+-------------------+-----------+---------+-------------+--------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|features|
+---------------+-------------------+-----------+---------+-------------+--------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|  [0.57]|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|  [0.47]|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|  [0.56]|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|  [0.57]|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|  [0.35]|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|  [0.41]|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|  [0.57]|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|  [0.41]|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|  [0.53]|
|   

In [18]:
from pyspark.ml.regression import LinearRegression

In [19]:
lr = LinearRegression(featuresCol='features', labelCol='session_count')

In [20]:
lrmodel = lr.fit(dfvectorutil)

22/03/26 10:09:46 WARN Instrumentation: [39fa4088] regParam is zero, which might cause numerical instability and overfitting.
22/03/26 10:09:48 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/03/26 10:09:48 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/03/26 10:09:58 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [21]:
lrmodel.coefficients 

DenseVector([47.024])

In [22]:
lrmodel.intercept 

40.416951035523695

In [23]:
lrmodel.summary.rootMeanSquaredError

12.83799022593174